In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from gensim.models import Word2Vec

# Download NLTK data
nltk.download('punkt')

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Define the file path to the dataset in Google Drive
file_path = '/content/drive/My Drive/train-balanced-sarcasm.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

# Reduce the dataset size to 1% for faster processing
df = df.sample(frac=0.01, random_state=42)

# Removing NaN Values
df = df.dropna(subset=['comment'])

# Ensure all comments are strings and handle NaN values
df['comment'] = df['comment'].astype(str)
df['comment'] = df['comment'].apply(lambda x: x if pd.notnull(x) else '')

# Preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    return tokens

df['tokens'] = df['comment'].apply(preprocess_text)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")

# Create a dictionary mapping words to their embeddings
embedding_index = {}
for word in word2vec_model.wv.index_to_key:
    embedding_index[word] = word2vec_model.wv[word]

# Prepare embedding matrix
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
sequences = tokenizer.texts_to_sequences(df['tokens'])
word_index = tokenizer.word_index
max_sequence_length = max(len(x) for x in sequences)

data = pad_sequences(sequences, maxlen=max_sequence_length)

# Create embedding matrix
embedding_dim = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['label'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_sequence_length,
                    trainable=False))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

# Classification report
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5
253/253 [==============================] - 76s 279ms/step - loss: 0.6926 - accuracy: 0.5037 - val_loss: 0.6889 - val_accuracy: 0.4763
Epoch 2/5
253/253 [==============================] - 70s 277ms/step - loss: 0.6901 - accuracy: 0.5016 - val_loss: 0.6889 - val_accuracy: 0.5574
Epoch 3/5
253/253 [==============================] - 65s 259ms/step - loss: 0.6898 - accuracy: 0.5195 - val_loss: 0.6883 - val_accuracy: 0.5589
Epoch 4/5
253/253 [==============================] - 66s 262ms/step - loss: 0.6889 - accuracy: 0.5262 - val_loss: 0.6866 - val_accuracy: 0.5539
Epoch 5/5
64/64 [==============================] - 5s 78ms/step - loss: 0.6879 - accuracy: 0.5569
Accuracy: 0.5568743944168091
64/64 [==============================] - 7s 92ms/step
              precision    recall  f1-score   support

           0       0.67      0.14      0.23       963
     